# GeometryDiffDevelopment

#### String difference computation
    
Method 1: difflib

    import difflib

    diff1 = [difflib.ndiff(g.to_wkt().split(','), h.to_wkt().split(',')) for g, h in zip(gdf1['Geometry'],gdf1b['Geometry'])]

    for i,s in enumerate(diff1[52]):
        if s[0]==' ': continue
        print(f'{i} {s[0]} {s[-1]}')

Method 2: diff-match-patch (from Google, more optimized)

In [ ]:
def condenseDiffPoints(diff,strip=False,usesets=True):
    '''Extracts changed lines from diff and cancels equivalent additions and subtractions.
    
    Parameters
    ----------
    diff: list of str, the result of a line difference (difflib.ndiff())
    split: bool, if True, strips whitespace from both ends of coordinate pairs.
    usesets: bool, if True, perform cancelling using sets instead of lists (eliminates duplicates)
    
    Returns
    -------
    list of strings, the lines of diff comprising changes that do not appear as both additions and subtractions
    '''
    addlist = [d[max(2,d.rfind("(")+1):(len(d) if d.find(")")==-1 else d.find(")"))] for d in diff if d[0]=='+']
    sublist = [d[max(2,d.rfind("(")+1):(len(d) if d.find(")")==-1 else d.find(")"))] for d in diff if d[0]=='-']
    # altlist = [d[max(2,d.rfind("(")+1):(len(d) if d.find(")")==-1 else d.find(")"))] for d in diff if d[0]=='?']
    if strip:
        addlist = [a.strip() for a in addlist]
        sublist = [s.strip() for s in sublist]
    if usesets:
        addlist = set(addlist)
        sublist = set(sublist)
        addlist_short = addlist.difference(sublist)
        sublist_short = sublist.difference(addlist)
    else:
        addlist_short = []
        sublist_short = sublist.copy()
        for i in range(len(addlist)):
            if addlist[i] in sublist_short:
                sublist_short.remove(addlist[i])
            else:
                addlist_short.append(addlist[i])
    addlist_short = ['+ '+a for a in addlist_short]
    sublist_short = ['- '+s for s in sublist_short]
    #if addlist_short==[] and sublist_short==[]:
    #    return [*addlist_short, *sublist_short]
    return [*addlist_short, *sublist_short]#,*altlist]

In [ ]:
def condenseDiffPoints(diff,strip=False,usesets=True):
    '''Extracts changed lines from diff and cancels equivalent additions and subtractions.
    
    Parameters
    ----------
    diff: list of str, the result of a line difference (difflib.ndiff())
    split: bool, if True, strips whitespace from both ends of coordinate pairs
    usesets: bool, if True, perform cancelling using sets instead of lists (eliminates duplicates)
    
    Returns
    -------
    list of strings, the lines of diff comprising changes that do not appear as both additions and subtractions
    '''
    diff = np.array(diff.copy())
    addind = list(np.where([d[0]=='+' for d in diff])[0])
    subind = list(np.where([d[0]=='-' for d in diff])[0])
    altind = list(np.where([d[0]=='?' for d in diff])[0])

    addind_alt = [i for i in addind if (i+1) in altind]
    subind_alt = [i for i in subind if (i+1) in altind]
    assert len([i for i in altind if not(((i-1) in addind) or ((i-1) in subind))])==0 # if difflib.ndiff() functions properly, '?' only follows
    
    addind_lon = [i for i in addind if not (i in addind_alt)]
    subind_lon = [i for i in subind if not (i in subind_alt)]

    # TODO replace these with a single stripped difflist, just use the appropriate index
    difflist = [d[max(2,d.rfind("(")+1):(len(d) if d.find(")")==-1 else d.find(")"))] for d in diff[addind]]
    addlist = difflist[addind]
    sublist = difflist[subind]
    altlist = diff[altind]
    if strip:
        addlist = [a.strip() for a in addlist]
        sublist = [s.strip() for s in sublist]
    if usesets:
        addlist = set(addlist)
        sublist = set(sublist)
        addlist_short = addlist.difference(sublist)
        sublist_short = sublist.difference(addlist)
    else:
        addlist_short = []
        addind_short = []
        sublist_short = sublist.copy()
        sublist_short = list(np.array(sublist_short)[[list(subind).index(i) for i in subind_lon]])
        subind_short = subind_lon.copy()
#        print(addind_alt,subind_alt,altind_alt,addind_short,subind_short)
        for ind, val in zip(addind, addlist):
            if ind in addind_lon:
                if val in sublist_short:
                    sind = sublist_short.index(val)
                    sublist_short.remove(val)
                    subind_short.remove(subind_short[sind])
                else:
                    addlist_short.append(val)
                    addind_short.append(ind)
    
    # Reconstruct list from _alt and _short indices into diff
#    print(addind_alt,subind_alt,altind_alt,addind_short,subind_short)
    rec_ind = addind_alt + subind_alt + altind_alt + altind_lon + addind_short + subind_short
    rec_ind.sort()
    return list(diff[rec_ind])#, list(rec_ind)
        
#    addlist_short = ['+ '+a for a in addlist_short]
#    sublist_short = ['- '+s for s in sublist_short]
#    if usesets or (addlist_short==[] and sublist_short==[]):
#        return [*addlist_short, *sublist_short]
#    return [*addlist_short, *sublist_short, *altlist]

In [ ]:
# Alternate ways to test if all indices of buffind_changed2 can be found in buffind_diff2
len(np.setdiff1d(buffind_changed2,buffind_diff2))==0
all([i in buffind_diff2 for i in buffind_changed2])
set(buffind_changed2).issubset(buffind_diff2)